## File Monologue

Create monologue from scratch

In [1]:
import os
from gai.lib.tests import make_local_tmp
here = make_local_tmp()

# Delete and create a new standard monologue file
from gai.messages.monologue import FileMonologue
agent_name = "TestBot"
file_path = os.path.join(here, f"{agent_name}.json")
monologue = FileMonologue(agent_name=agent_name,file_path=file_path)

# Check if standard monologue id exists
os.path.exists(file_path)

messages = monologue.list_messages()
assert len(messages) == 0


Add user message


In [2]:
# Add user message
from types import SimpleNamespace
mock_state = SimpleNamespace(state="GENERATE", input={"step":5}, title="GENERATE")
monologue.add_user_message("Hello, how are you?", mock_state)
messages = monologue.list_messages()
assert len(messages) == 1
assert messages[0].body.role == "user"
assert messages[0].body.content == "Hello, how are you?"
print("✓ User message added correctly")

✓ User message added correctly


Add assistant message

In [3]:
# Add assistant message
mock_state = SimpleNamespace(state="GENERATE", input={"step":6}, title="GENERATE")
monologue.add_assistant_message("I'm doing well, thank you!", mock_state)
messages = monologue.list_messages()
assert len(messages) == 2
assert messages[1].body.role == "assistant"
assert messages[1].body.content == "I'm doing well, thank you!"
print("✓ Assistant message added correctly")

✓ Assistant message added correctly


List Messages

In [4]:
messages = monologue.list_messages()
assert len(messages) == 2

Pop Message

In [5]:
messages = monologue.list_messages()
last_message_id = messages[-1].id
assert any(m.id == last_message_id for m in messages)

monologue.pop()
messages = monologue.list_messages()
assert not any(m.id == last_message_id for m in messages)

Create monologue from messages

In [1]:
from gai.messages import MessagePydantic
messages = [
    MessagePydantic(**{
        'id': 'b1e5f98c-f6eb-47de-a6e2-387510d970f9', 
        'header': {
            'sender': 'User',
            'recipient': 'Assistant',
            "timestamp": 1751308157.270983,
            "order": 0            
        }, 'body': {
            'type': 'monologue',
            "state_name": "CHAT",
            "step_no": 1,
            "content_type":"text",
            'role': "user",
            'content': 'Tell me a one paragragh story.',
        }
    }),
    MessagePydantic(**{
        'id': 'abbc7961-45dc-4973-aaf4-a6224ed35d37', 
        'header': {
            'sender': 'Assistant',
            'recipient': 'User',
            "timestamp": 1751308167.3488164,
            "order": 1
        }, 'body': {
            'type': 'monologue',
            "state_name": "CHAT",
            "step_no": 2,
            "content_type":"text",
            'role': "assistant",
            'content': [
                {
                    "citations":None,
                    "text": "I will search for a story online.",
                },
                {
                    "id": "toolu_013epBSqLbV61tRCLd1aMT86",
                    "input": {
                        "url": "https://www.story.com"
                    },
                    "name": "scrape",
                    "type": "tool_use"                    
                }
            ]
        }
    }),
    MessagePydantic(**{
        "id": "f22fd2b5-dc7e-4389-a079-703d28dd365a",
        "header": {
            "sender": "User",
            "recipient": "Assistant",
            "timestamp": 1751308180.9790845,
            "order": 2
        },
        "body": {
            "type": "monologue",
            "state_name": "TOOL_USE",
            "step_no": 3,
            "content_type": "text",
            "role": "user",
            "content": [
                {
                    "type": "tool_result",
                    "tool_use_id": "toolu_013epBSqLbV61tRCLd1aMT86",
                    "content": "Once upon a time, in a small village, there lived a kind..."
                }
            ]
        }
    }),
    MessagePydantic(**{
        'id': '787ef108-16f9-489a-a6f1-141aff5f0e49',
        'header': {
            'sender': 'Assistant',
            'recipient': 'User',
            "timestamp": 1751308181.9790845,
            "order": 3
        }, 'body': {
            'type': 'monologue',
            "state_name": "TOOL_USE",
            "step_no": 4,
            "content_type":"text",
            'role': "assistant",
            'content':"Once upon a time, in a small village, there lived a kind..."
        }
    }),
]

# Create monologue
import os
from gai.lib.tests import make_local_tmp
here = make_local_tmp()
from gai.messages.monologue import FileMonologue
file_path = os.path.join(here,"monologue.json")
monologue = FileMonologue(agent_name="TestBot",file_path=file_path,messages=messages)
messages = monologue.list_messages()

assert len(messages) == 4
assert messages[0].body.role == "user"
assert messages[0].body.content == "Tell me a one paragragh story."
assert messages[1].body.role == "assistant"
assert messages[1].body.content[0]["text"] == "I will search for a story online."
assert messages[1].body.content[1]["type"] == "tool_use"
assert messages[2].body.role == "user"
assert messages[2].body.content[0]["type"] == "tool_result"
assert messages[3].body.role == "assistant"
assert messages[3].body.content == "Once upon a time, in a small village, there lived a kind..."
print("✓ Batch messages added correctly")


✓ Batch messages added correctly


### get_last_toolcalls

This is to get the tool calls from the last message in the monologue.

If there are no tool calls, it will return an empty list.

In [5]:
monologue.get_last_toolcalls()

[]

If last message is an assistant message and contains "type":"tool_use", it will return the tool calls.

In [3]:
monologue.add_assistant_message(
    content=[
        {
            "citations":None,
            "text": "I will search for a story online.",
        },
        {
            "id": "toolu_013epBSqLbV61tRCLd1aMT86",
            "input": {
                "url": "https://www.story.com"
            },
            "name": "scrape",
            "type": "tool_use"                    
        }        
    ]
)
monologue.get_last_toolcalls()

[{'tool_use_id': 'toolu_013epBSqLbV61tRCLd1aMT86',
  'tool_name': 'scrape',
  'arguments': {'url': 'https://www.story.com'}}]